## Importing python libraries 

In [65]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer

### Loading Dataset 

In [66]:
df_mobile_price = pd.read_csv('Processed_Flipdata - Processed_Flipdata.csv')
df_mobile_price.head()
df_mobile_price.columns

Index(['Unnamed: 0', 'Model', 'Colour', 'Memory', 'RAM', 'Battery_',
       'Rear Camera', 'Front Camera', 'AI Lens', 'Mobile Height', 'Processor_',
       'Prize'],
      dtype='object')

## Dropping 'Unnamed: 0' column from the given dataset

In [67]:
df_mobile_price=df_mobile_price.drop(columns=['Unnamed: 0'])  ## dropping 'Unnamed: 0' column
df_mobile_price.head()   ## checking dataset after dropping 'Unnamed: 0' column

,Model,Colour,Memory,RAM,Battery_,Rear Camera,Front Camera,AI Lens,Mobile Height,Processor_,Prize
0,Infinix SMART 7,Night Black,64,4,6000,13MP,5MP,1,16.76,Unisoc Spreadtrum SC9863A1,"7,299"
1,Infinix SMART 7,Azure Blue,64,4,6000,13MP,5MP,1,16.76,Unisoc Spreadtrum SC9863A1,"7,299"
2,MOTOROLA G32,Mineral Gray,128,8,5000,50MP,16MP,0,16.64,Qualcomm Snapdragon 680,"11,999"
3,POCO C50,Royal Blue,32,2,5000,8MP,5MP,0,16.56,Mediatek Helio A22,"5,649"
4,Infinix HOT 30i,Marigold,128,8,5000,50MP,5MP,1,16.76,G37,"8,999"


## Shape of original given dataset after dropping 'Unnamed: 0' column

In [68]:
shape = df_mobile_price.shape
print(f"Number of rows in given dataset are {shape[0]} and columns are {shape[1]}")

Number of rows in given dataset are 541 and columns are 11


### Dataset Info

In [69]:
df_mobile_price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541 entries, 0 to 540
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Model          541 non-null    object 
 1   Colour         541 non-null    object 
 2   Memory         541 non-null    int64  
 3   RAM            541 non-null    int64  
 4   Battery_       541 non-null    int64  
 5   Rear Camera    541 non-null    object 
 6   Front Camera   541 non-null    object 
 7   AI Lens        541 non-null    int64  
 8   Mobile Height  541 non-null    float64
 9   Processor_     541 non-null    object 
 10  Prize          541 non-null    object 
dtypes: float64(1), int64(4), object(6)
memory usage: 46.6+ KB


Null value not present in any column

## Cleaning of data

In [70]:
df_mobile_price.drop_duplicates  ## dropping duplicates
df_mobile_price.shape ## checking shape of dataset after removing duplicates. 

(541, 11)

## Creating New Columns - Brand_Name and Model_Name

In [71]:
df_mobile_price['Brand_Name'] = df_mobile_price['Model'].astype("string").str.split().str[0]  ## spliting Model into Brand_Name and Model_Name
df_mobile_price['Model_Name'] = df_mobile_price['Model'].astype("string").str.split(n=1).str[1]     


In [72]:
df_mobile_price.head(3)

,Model,Colour,Memory,RAM,Battery_,Rear Camera,Front Camera,AI Lens,Mobile Height,Processor_,Prize,Brand_Name,Model_Name
0,Infinix SMART 7,Night Black,64,4,6000,13MP,5MP,1,16.76,Unisoc Spreadtrum SC9863A1,"7,299",Infinix,SMART 7
1,Infinix SMART 7,Azure Blue,64,4,6000,13MP,5MP,1,16.76,Unisoc Spreadtrum SC9863A1,"7,299",Infinix,SMART 7
2,MOTOROLA G32,Mineral Gray,128,8,5000,50MP,16MP,0,16.64,Qualcomm Snapdragon 680,"11,999",MOTOROLA,G32


In [73]:
df_mobile_price['Brand_Name'].value_counts() # checking count of each category

Brand_Name
realme       98
REDMI        68
Infinix      65
vivo         62
POCO         60
SAMSUNG      55
MOTOROLA     46
Tecno        17
OPPO         16
micromax      9
Google        9
redmi         5
OnePlus       5
APPLE         5
Micromax      4
Nokia         3
itel          3
IQOO          2
nothing       2
Redmi         2
I             2
LAVA          2
micromax1     1
Name: count, dtype: int64

In [74]:
df_mobile_price['Model_Name'].value_counts() # checking count of each category

Model_Name
C55               14
10                11
Note 12 Pro 5G    11
Galaxy A14 5G      9
Y16                9
                  ..
Galaxy A04e        1
Galaxy S23 5G      1
Z21                1
Spark 8T           1
A77                1
Name: count, Length: 176, dtype: int64

## Re-arranging Brand_Name and Model_Name columns and Deleting Model column

In [75]:
col = df_mobile_price.pop('Brand_Name')    # pop the Brand_Name column and stored in col variable
df_mobile_price.insert(0,'Brand_Name',col) # insert the Brand_Name column at 0 index

In [76]:
col1 = df_mobile_price.pop('Model_Name') ## pop the Model_Name column and stored in col variable
df_mobile_price.insert(1,'Model_Name',col1) ### insert the Model_Name column at 1 index

In [77]:
df_mobile_price = df_mobile_price.drop(columns=(['Model']))   ## Dropping Model column

In [78]:
df_mobile_price.head()

,Brand_Name,Model_Name,Colour,Memory,RAM,Battery_,Rear Camera,Front Camera,AI Lens,Mobile Height,Processor_,Prize
0,Infinix,SMART 7,Night Black,64,4,6000,13MP,5MP,1,16.76,Unisoc Spreadtrum SC9863A1,"7,299"
1,Infinix,SMART 7,Azure Blue,64,4,6000,13MP,5MP,1,16.76,Unisoc Spreadtrum SC9863A1,"7,299"
2,MOTOROLA,G32,Mineral Gray,128,8,5000,50MP,16MP,0,16.64,Qualcomm Snapdragon 680,"11,999"
3,POCO,C50,Royal Blue,32,2,5000,8MP,5MP,0,16.56,Mediatek Helio A22,"5,649"
4,Infinix,HOT 30i,Marigold,128,8,5000,50MP,5MP,1,16.76,G37,"8,999"


## Moving Snapdragon and Qualcomm values to Prossor_Vendor Column and converting all those values to Qualcomm

In [79]:
df_mobile_price['Processor_Vendor'] = pd.NA

In [80]:

s = df_mobile_price['Processor_'].astype(str)

mask = (
    s.str.contains("snapdragon", case = False, na = False) | 
    s.str.contains("qualcomm", case = False, na = False) 
)


In [81]:

df_mobile_price.loc[mask,'Processor_Vendor'] = 'Qualcomm'



In [82]:
print("Null values for column Processor_Vendor:",df_mobile_price['Processor_Vendor'].isnull().sum())   # checking null values.
print("\n")
print(df_mobile_price['Processor_Vendor'].value_counts())   # checking category values.

Null values for column Processor_Vendor: 411


Processor_Vendor
Qualcomm    130
Name: count, dtype: int64


## Moving ios, IOS and 6 Core values to Prossor_Vendor Column and converting all those values to Apple

In [83]:
s = df_mobile_price['Processor_'].astype("string")

mask = (
    s.str.contains('ios',case=False,na=False) |
    s.str.contains('6core',case=False,na=False)

)

In [84]:
df_mobile_price.loc[mask,'Processor_Vendor'] = 'Apple'

In [85]:
print("Null values for column Processor_Vendor:",df_mobile_price['Processor_Vendor'].isnull().sum())   # checking null values.
print("\n")
print(df_mobile_price['Processor_Vendor'].value_counts())   # checking category values.

Null values for column Processor_Vendor: 407


Processor_Vendor
Qualcomm    130
Apple         4
Name: count, dtype: int64


## Moving Google Tensor, Tensor G2 and Google Tensor G2 values to Prossor_Vendor Column and converting all those values to Google

In [86]:
s = df_mobile_price['Processor_'].astype("string")

mask = (
    s.str.contains('Tensor',case=False,na=False)
)

In [87]:
df_mobile_price.loc[mask,'Processor_Vendor'] = 'Google'

In [88]:
print("Null values for column Processor_Vendor:",df_mobile_price['Processor_Vendor'].isnull().sum())   # checking null values.
print("\n")
print(df_mobile_price['Processor_Vendor'].value_counts())   # checking category values.

Null values for column Processor_Vendor: 399


Processor_Vendor
Qualcomm    130
Google        8
Apple         4
Name: count, dtype: int64


## Moving mediatek, mediatek, helio ,dimensity ,mtk and mt values to Prossor_Vendor Column and converting all those values to MediaTek

In [89]:
s = df_mobile_price['Processor_'].astype("string")

mask = (
    s.str.contains('mediatek',case=False,na=False)|
    s.str.contains('medaitek',case=False,na=False)|
    s.str.contains('helio',case=False,na=False)|
    s.str.contains('dimensity',case=False,na=False)|
    s.str.contains('mtk',case=False,na=False)|
    s.str.contains('mtk',case=False,na=False)

)

In [90]:
df_mobile_price.loc[mask,'Processor_Vendor'] = 'MediaTek'

In [91]:
print("Null values for column Processor_Vendor:",df_mobile_price['Processor_Vendor'].isnull().sum())   # checking null values.
print("\n")
print(df_mobile_price['Processor_Vendor'].value_counts())   # checking category values.

Null values for column Processor_Vendor: 123


Processor_Vendor
MediaTek    276
Qualcomm    130
Google        8
Apple         4
Name: count, dtype: int64


## Moving unisoc, spreadtrum and sc values to Prossor_Vendor Column and converting all those values to Unisoc

In [92]:
s = df_mobile_price['Processor_'].astype("string")

mask = (
    s.str.contains('unisoc',case=False,na=False)|
    s.str.contains('spreadtrum',case=False,na=False)|
    s.str.contains('uni',case=False,na=False)
)

In [93]:
df_mobile_price.loc[mask,'Processor_Vendor'] = 'Unisoc'

In [94]:
print("Null values for column Processor_Vendor:",df_mobile_price['Processor_Vendor'].isnull().sum())   # checking null values.
print("\n")
print(df_mobile_price['Processor_Vendor'].value_counts())   # checking category values.

Null values for column Processor_Vendor: 62


Processor_Vendor
MediaTek    276
Qualcomm    130
Unisoc       61
Google        8
Apple         4
Name: count, dtype: int64


## Moving samsung exynos or exynos to Prossor_Vendor Column and converting all those values to Samsung

In [95]:
s = df_mobile_price['Processor_'].astype("string")

mask = (
    s.str.contains('samsung',case=False,na=False)|
    s.str.contains('exynos',case=False,na=False)
)

In [96]:
df_mobile_price.loc[mask,'Processor_Vendor'] = 'Samsung'

In [97]:
print("Null values for column Processor_Vendor:",df_mobile_price['Processor_Vendor'].isnull().sum())   # checking null values.
print("\n")
print(df_mobile_price['Processor_Vendor'].value_counts())   # checking category values.

Null values for column Processor_Vendor: 32


Processor_Vendor
MediaTek    276
Qualcomm    130
Unisoc       61
Samsung      30
Google        8
Apple         4
Name: count, dtype: int64


## Remaining 32 entries which have incomplete processor name or without vendor name or half information marked them as other/unknown

In [98]:
df_mobile_price['Processor_Vendor'] = df_mobile_price['Processor_Vendor'].replace(np.nan,'Other/Unknown')

In [99]:
print("Null values for column Processor_Vendor:",df_mobile_price['Processor_Vendor'].isnull().sum())   # checking null values.
print("\n")
print(df_mobile_price['Processor_Vendor'].value_counts())   # checking category values.
print("\n")
print(df_mobile_price['Processor_Vendor'].unique()) #checking unique values.

Null values for column Processor_Vendor: 0


Processor_Vendor
MediaTek         276
Qualcomm         130
Unisoc            61
Other/Unknown     32
Samsung           30
Google             8
Apple              4
Name: count, dtype: int64


['Unisoc' 'Qualcomm' 'MediaTek' 'Other/Unknown' 'Samsung' 'Google' 'Apple']


In [100]:
df_mobile_price.head(10)

,Brand_Name,Model_Name,Colour,Memory,RAM,Battery_,Rear Camera,Front Camera,AI Lens,Mobile Height,Processor_,Prize,Processor_Vendor
0,Infinix,SMART 7,Night Black,64,4,6000,13MP,5MP,1,16.76,Unisoc Spreadtrum SC9863A1,"7,299",Unisoc
1,Infinix,SMART 7,Azure Blue,64,4,6000,13MP,5MP,1,16.76,Unisoc Spreadtrum SC9863A1,"7,299",Unisoc
2,MOTOROLA,G32,Mineral Gray,128,8,5000,50MP,16MP,0,16.64,Qualcomm Snapdragon 680,"11,999",Qualcomm
3,POCO,C50,Royal Blue,32,2,5000,8MP,5MP,0,16.56,Mediatek Helio A22,"5,649",MediaTek
4,Infinix,HOT 30i,Marigold,128,8,5000,50MP,5MP,1,16.76,G37,"8,999",Other/Unknown
5,Infinix,HOT 30i,Glacier Blue,128,8,5000,50MP,5MP,1,16.76,G37,"8,999",Other/Unknown
6,Infinix,HOT 30i,Mirror Black,128,8,5000,50MP,5MP,1,16.76,G37,"8,999",Other/Unknown
7,Infinix,HOT 30i,Diamond White,128,8,5000,50MP,5MP,1,16.76,G37,"8,999",Other/Unknown
8,Infinix,HOT 30i,Diamond White,64,4,5000,50MP,5MP,1,16.76,G37,"7,999",Other/Unknown
9,Infinix,HOT 30i,Mirror Black,64,4,5000,50MP,5MP,1,16.76,G37,"7,999",Other/Unknown


In [101]:
df_mobile_price['Colour'].value_counts()

Colour
Yellow            12
Black              8
Power Black        8
Royal Blue         8
Midnight Black     7
                  ..
Flame Red          1
Oxygen Green       1
Cyan               1
Awesome Lime       1
Sky Blue           1
Name: count, Length: 275, dtype: int64

In [102]:
price = df_mobile_price[(df_mobile_price['Colour'].isin(['Yellow','Black','Power Black','Royal Blue','Midnight Black'])) & (df_mobile_price['Brand_Name']=='SAMSUNG')]
price.sort_values(by='Prize',ascending=True)

,Brand_Name,Model_Name,Colour,Memory,RAM,Battery_,Rear Camera,Front Camera,AI Lens,Mobile Height,Processor_,Prize,Processor_Vendor
413,SAMSUNG,Galaxy A14 5G,Black,64,4,5000,50MP,13MP,0,16.76,SEC S5E8535 (Exynos 1330),"16,499",Samsung
427,SAMSUNG,Galaxy A23,Black,128,6,5000,50MP,8MP,0,16.76,Octa-core(EXYNOS),"17,499",Samsung
318,SAMSUNG,Galaxy A14 5G,Black,128,6,5000,50MP,13MP,0,16.76,SEC S5E8535 (Exynos 1330),"18,999",Samsung
425,SAMSUNG,Galaxy A14 5G,Black,128,8,5000,50MP,13MP,0,16.76,SEC S5E8535 (Exynos 1330),"20,999",Samsung
